In [2]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845499 sha256=efab3a2efef9479e718cc2875d65e27ba396457347987341a9a82a6100799c3b
  Stored in directory: /Users/huanglijie/Library/Caches/pip/wheels/51/c8/18/298a4ced8ebb3ab8a7d26a7198c0cc7035abb906bde94a4c4b
Successfully built pyspark


In [26]:
# 读取数据

spark = SparkSession.builder.config(conf = SparkConf()).getOrCreate()

data = spark.read.format("csv").option("header", "true").load("./input/overdue_details.csv")
data


DataFrame[aid: string, overdue_freq: string, overdue_times: string, ID: string, CODE_GENDER: string, FLAG_OWN_CAR: string, FLAG_OWN_REALTY: string, CNT_CHILDREN: string, AMT_INCOME_TOTAL: string, NAME_INCOME_TYPE: string, NAME_EDUCATION_TYPE: string, NAME_FAMILY_STATUS: string, NAME_HOUSING_TYPE: string, DAYS_BIRTH: string, DAYS_EMPLOYED: string, FLAG_MOBIL: string, FLAG_WORK_PHONE: string, FLAG_PHONE: string, FLAG_EMAIL: string, OCCUPATION_TYPE: string, CNT_FAM_MEMBERS: string]

In [10]:
all_list = []
total_y = [] # overall number
total_y.append(data.filter(data['overdue_times'] == 0).count())
total_y.append(data.filter(data['overdue_times'] > 0).count())
all_list.append(total_y)

In [12]:
from pyecharts.charts import Bar
from pyecharts.charts import Pie
from pyecharts.charts import Page
from pyecharts import options as opts

In [22]:
total_list = all_list[0]
attr = ["num of people paid in time", "num of people overdue"]
pie = (
    Pie()
        .add("Overall Number", [list(z) for z in zip(attr,total_list)])
        .set_global_opts(title_opts=opts.TitleOpts(title="Overall Number"))
        .set_series_opts(
        tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"),
        label_opts=opts.LabelOpts(formatter="{b}: {c} ({d}%)")
        )
)
total_pie = pie
page = Page()
page.add(total_pie)
page.render('overall_bi_pie.html')

'/Users/huanglijie/Documents/Columbia_University/1st_semeter/EECS6893_BIG_DATA_ANALYTICS/group_project/overall_bi_pie.html'

In [38]:
data = data.withColumn("overdue_freq",data.overdue_freq.astype("float"))
total_freq = []
total_freq.append(data.filter(data['overdue_freq'] == 0).count())
total_freq.append(data.filter(data['overdue_freq'] > 0).filter(data['overdue_freq'] < 0.2).count())
total_freq.append(data.filter(data['overdue_freq'] >= 0.2).filter(data['overdue_freq'] < 0.4).count())
total_freq.append(data.filter(data['overdue_freq'] >= 0.4).filter(data['overdue_freq'] < 0.6).count())
total_freq.append(data.filter(data['overdue_freq'] >= 0.6).filter(data['overdue_freq'] < 0.8).count())
total_freq.append(data.filter(data['overdue_freq'] >= 0.8).filter(data['overdue_freq'] < 1).count())
total_freq.append(data.filter(data['overdue_freq'] == 1).count())

all_list.append(total_freq)

total_freq_list = all_list[1]
attr = ["Paid in Time", "Overdue Less than 20%", "20% ~ 40%", "40% ~ 60%", "60% ~ 80%", "80% ~ 100%", "100%"]
pie = (
    Pie()
        .add("Overall Freq", [list(z) for z in zip(attr,total_freq_list)])
        .set_global_opts(title_opts=opts.TitleOpts(title="Overall Overdue Frequence"))
        .set_series_opts(
        tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"),
        label_opts=opts.LabelOpts(formatter="{b}: {c} ({d}%)")
        )
)
total_pie = pie
page.add(total_pie)
page.render('overall_pie.html')

'/Users/huanglijie/Documents/Columbia_University/1st_semeter/EECS6893_BIG_DATA_ANALYTICS/group_project/overall_pie.html'